In [26]:
import pandas as pd
import numpy as np

from sklearn import preprocessing   # Scaling

from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae

from statsmodels.tsa.stattools import pacf

## Neural Network

from keras.models import Sequential    # For linear stacking of layers

from keras.optimizers import SGD    # Optimization: sgd stands for stochastic gradient descent

from keras.layers.core import Dense, Activation    # An Activation fuction and a Dense layer for output layer.

from keras.layers.recurrent import SimpleRNN    # A fully connected RNN where the output is to be fed to input layer.

In [6]:
temp = pd.read_csv('google.csv')

In [7]:
temp.head()

,date,open,high,low,volume,close,direction
0,03-01-2007,231.494354,236.789917,229.065155,15513200.0,232.284210,0
1,04-01-2007,232.984665,240.411362,232.661758,15877700.0,240.068588,1
2,05-01-2007,239.691040,242.174881,237.510223,13833500.0,242.020889,1
3,08-01-2007,242.269272,243.352234,239.542007,9570600.0,240.227554,0
4,09-01-2007,241.156509,242.547470,239.045242,10832700.0,241.181351,1


In [8]:
temp.tail()

,date,open,high,low,volume,close,direction
2825,23-03-2018,1047.030029,1063.359985,1021.219971,2156700.0,1021.570007,0
2826,26-03-2018,1046.000000,1055.630005,1008.400024,2665100.0,1053.209961,1
2827,27-03-2018,1063.000000,1064.838989,996.919983,3095300.0,1005.099976,0
2828,28-03-2018,998.000000,1024.229980,980.640015,3369300.0,1004.559998,0
2829,29-03-2018,1011.630005,1043.000000,1002.900024,2702200.0,1031.790039,1


In [9]:
## Dropping Date Column
data = temp.drop(temp.columns[[0]], axis=1)

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2830 entries, 0 to 2829
Data columns (total 6 columns):
open         2830 non-null float64
high         2830 non-null float64
low          2830 non-null float64
volume       2830 non-null float64
close        2830 non-null float64
direction    2830 non-null int64
dtypes: float64(5), int64(1)
memory usage: 132.7 KB


In [11]:
target = data['close']

In [12]:
x = data.drop(data.columns[[4,5]], axis=1)

x.head()

,open,high,low,volume
0,231.494354,236.789917,229.065155,15513200.0
1,232.984665,240.411362,232.661758,15877700.0
2,239.691040,242.174881,237.510223,13833500.0
3,242.269272,243.352234,239.542007,9570600.0
4,241.156509,242.547470,239.045242,10832700.0


# Data Preprocessing

## Transformations

- Step 1: Apply Log transformation
- Step 2: Reshape Array as a 2-D so that we can apply scaler function
- Step 3: Reshape Array to 1-D

In [13]:
## Step 1:
x = x.apply(np.log)

x = pd.concat([x, data['direction']], axis=1)

In [15]:
scaler_x = preprocessing.MinMaxScaler(feature_range=(0,1))    # Rescaling the parameters

scaler_y = preprocessing.MinMaxScaler(feature_range=(0,1))    # Rescaling the parameters

In [16]:
### Reshape

# In order to use fit_transform we need to reshape 'x' and 'y' to 2-D Array

x = np.array(x).reshape((len(x), 5))

x = scaler_x.fit_transform(x)

y = np.array(target).reshape((len(target),1))

y = np.log(y)

y = scaler_y.fit_transform(y)

## Assessing Partial Auto-Correlation

- A Partial Auto-Correlation is the amount of Linear Correlation between an observation x<sub>t</sub> and a lag of itself (say x<sub>t-k</sub> ) that is **not explained by correlations of the obervations in between.**

In [ ]:
x_pacf = pacf(pacf1, nlags=6, method='ols')

print(x_pacf)

#### Train- Test Division

In [59]:
end =  len(x) -1 
learn_end = int(end*0.95)

x_train = x[0:learn_end - 1,]

x_test = x[learn_end:end-1, ]

In [65]:
y_train = y[1:learn_end]

y_test = y[learn_end+1:end]

In [70]:
### Reshape for Keras

x_train = x_train.reshape(x_train.shape + (1,))

x_test = x_test.reshape(x_test.shape + (1,))

### Modeling

In [82]:
### Model Structure

seed = 666
np.random.seed(seed)

fit1 = Sequential()

### Keras 2 API: output_dim is changed to units
fit1.add(SimpleRNN(units =5, activation = "tanh", input_shape = (5,1)))
fit1.add(Dense(units=1, activation = "linear"))

In [83]:
#### Choosing the momentum

sgd = SGD(lr=0.0001, momentum=0.8, nesterov=True)

fit1.compile(loss="mean_squared_error", optimizer=sgd)

### Model Training

In [84]:
fit1.fit(x_train, y_train, batch_size=10, epochs=500)

Instructions for updating:
Use tf.cast instead.
Epoch 1/500
2686/2686 [==============================] - 1s 239us/step - loss: 0.1537
Epoch 2/500
2686/2686 [==============================] - 0s 105us/step - loss: 0.0559
Epoch 3/500
2686/2686 [==============================] - 0s 105us/step - loss: 0.0409
Epoch 4/500
2686/2686 [==============================] - 0s 106us/step - loss: 0.0303
Epoch 5/500
2686/2686 [==============================] - 0s 105us/step - loss: 0.0225
Epoch 6/500
2686/2686 [==============================] - 0s 105us/step - loss: 0.0169
Epoch 7/500
2686/2686 [==============================] - 0s 106us/step - loss: 0.0127
Epoch 8/500
2686/2686 [==============================] - 0s 105us/step - loss: 0.0097
Epoch 9/500
2686/2686 [==============================] - 0s 105us/step - loss: 0.0075
Epoch 10/500
2686/2686 [==============================] - 0s 105us/step - loss: 0.0059
Epoch 11/500
2686/2686 [==============================] - 0s 106us/step - loss: 0.0047
Epoc

2686/2686 [==============================] - 0s 97us/step - loss: 8.7011e-04
Epoch 184/500
2686/2686 [==============================] - 0s 95us/step - loss: 8.6804e-04
Epoch 185/500
2686/2686 [==============================] - 0s 94us/step - loss: 8.6504e-04
Epoch 186/500
2686/2686 [==============================] - 0s 95us/step - loss: 8.6335e-04
Epoch 187/500
2686/2686 [==============================] - 0s 95us/step - loss: 8.6107e-04
Epoch 188/500
2686/2686 [==============================] - 0s 95us/step - loss: 8.5852e-04
Epoch 189/500
2686/2686 [==============================] - 0s 95us/step - loss: 8.5609e-04
Epoch 190/500
2686/2686 [==============================] - 0s 96us/step - loss: 8.5376e-04
Epoch 191/500
2686/2686 [==============================] - 0s 104us/step - loss: 8.5121e-04
Epoch 192/500
2686/2686 [==============================] - 0s 105us/step - loss: 8.4866e-04
Epoch 193/500
2686/2686 [==============================] - 0s 94us/step - loss: 8.4709e-04
Epoch 194/5

2686/2686 [==============================] - 0s 100us/step - loss: 5.4331e-04
Epoch 362/500
2686/2686 [==============================] - 0s 106us/step - loss: 5.4182e-04
Epoch 363/500
2686/2686 [==============================] - 0s 100us/step - loss: 5.4037e-04
Epoch 364/500
2686/2686 [==============================] - 0s 99us/step - loss: 5.3888e-04
Epoch 365/500
2686/2686 [==============================] - 0s 100us/step - loss: 5.3767e-04
Epoch 366/500
2686/2686 [==============================] - 0s 97us/step - loss: 5.3623e-04
Epoch 367/500
2686/2686 [==============================] - 0s 97us/step - loss: 5.3513e-04
Epoch 368/500
2686/2686 [==============================] - 0s 97us/step - loss: 5.3361e-04
Epoch 369/500
2686/2686 [==============================] - 0s 98us/step - loss: 5.3233e-04
Epoch 370/500
2686/2686 [==============================] - 0s 97us/step - loss: 5.3089e-04
Epoch 371/500
2686/2686 [==============================] - 0s 98us/step - loss: 5.2986e-04
Epoch 372

In [85]:
score_train = fit1.evaluate(x_train, y_train, batch_size=10)

2686/2686 [==============================] - 0s 72us/step


In [86]:
score_test = fit1.evaluate(x_test, y_test, batch_size=10)

141/141 [==============================] - 0s 64us/step


### Prediction

In [90]:
pred1 =  fit1.predict(x_test)

pred1 = scaler_y.inverse_transform(np.array(pred1).reshape((len(pred1),1)))

In [91]:
pred1 = np.exp(pred1)

In [101]:
## Transforming to original value
y_true = scaler_y.inverse_transform(np.array(y_test).reshape((len(y_test),1)))

y_true = np.exp(y_true)

In [106]:
### Model Evaluation
np.sqrt(mse(y_true, pred1))

141.279618336662